In [1]:
import nltk   
import spacy            
import re     
import string            
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
from nltk.corpus import twitter_samples    # Corpus Twitter
from nltk.tokenize import word_tokenize 
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
nltk.download('stopwords')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


False

Lectura de Corpus

In [2]:
nltk.download('twitter_samples')

[nltk_data] Error loading twitter_samples: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


False

In [3]:
pos_tweets = twitter_samples.strings('positive_tweets.json') #tweets positivos
neg_tweets = twitter_samples.strings('negative_tweets.json') #tweets negativos

print("Positive tweets: ", len(pos_tweets))
print("Negative tweets: ", len(neg_tweets))

Positive tweets:  5000
Negative tweets:  5000


Procesamiento


1. LowerCase
2. Lematización / Stemming
3. Remover stopword
4. Remover signos de puntuación
4. Remover urls y manejadores





In [4]:
def custom_tokenizer(nlp):
    special_cases = {":)": [{"ORTH": ":)"}], ":(": [{"ORTH": ":("}]}
    simple_url_re = re.compile(r'''^https?://''')
    suffixes = nlp.Defaults.suffixes + [r'''-+$''',]
    prefixes = nlp.Defaults.prefixes + [r'^[\-\—\–\+\+\.\!\/\,\"\(\)\[\]\{\}\:\;\<\>\?\¿\¡\|\&\#\@\$\%\^\*\_\\\'\`\~]']
    suffix_regex = spacy.util.compile_suffix_regex(suffixes)
    prefixes_regex = spacy.util.compile_prefix_regex(prefixes)
    return spacy.tokenizer.Tokenizer(nlp.vocab, rules=special_cases, suffix_search=suffix_regex.search, prefix_search=prefixes_regex.search, url_match=simple_url_re.match)

nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = custom_tokenizer(nlp)


In [5]:
def normalization(data, regularization="lemma", language='english'):
  stopwords = nltk.corpus.stopwords.words(language)
  ps = PorterStemmer()
  normalized_data = []
  
  for tweet in data:
    tweet = re.sub(r'^RT[\s]+', '', tweet) # identificar retweets
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet) #eliminar links
    tweet = re.sub(r'#', '', tweet) #eliminar símbolo gato
    tweet = re.sub(r'@\w+', '', tweet) #eliminar palabras que inicias con @
    tweet = re.sub(r'\d+', '', tweet) #eliminar números
    tweet = re.sub(' +', ' ', tweet) #quitar espacios

    if regularization == "stem":
      tweetTokenizer = TweetTokenizer()
      words = tweetTokenizer.tokenize(tweet)
      tokens = [ps.stem(w) for w in words]
    if regularization == "lemma":
      doc = nlp(tweet)
      tokens = [token.lemma_ for token in doc]
    else:
      doc = nlp(tweet)
      tokens = [token.text for token in doc]
    
    normalized_tweets = [w for w in tokens if w not in stopwords and not w==' ' and w not in string.punctuation]
    normalized_data.append(normalized_tweets)
  return normalized_data

Create Vocabulary and frequency dictionaries

In [6]:
norm_pos = normalization(pos_tweets)
norm_neg = normalization(neg_tweets)
all_tweets = norm_pos + norm_neg
at = [w for tweet in all_tweets for w in tweet]
fd = nltk.FreqDist(at)
vocabulary = sorted(list(fd.keys()))

print('\nThe vocabulary has %d words.\n' %len(vocabulary))


The vocabulary has 12428 words.



In [7]:
X_features = []
for text in all_tweets:
  vector = [] # initialize in 1 ?
  for voc in vocabulary:
    # In vector saves a list of vocabulary's length. 
    # Iterate each vocabulary word and count in each text list
    vector.append(text.count(voc))
  X_features.append(vector)

print('\nX_features matrix has m = %d examples (rows).\n' %len(X_features))
print('and  n = %d features (columns).\n' %len(X_features[0]))


X_features matrix has m = 10000 examples (rows).

and  n = 12428 features (columns).



In [8]:
tags = [1]*len(pos_tweets) + [0]*len(neg_tweets)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_features,tags,test_size=0.2, random_state=50)
target_names = ['class 0', 'class 1']

In [14]:
# Crear un clasificador con DecisionTree
modeloDT = DecisionTreeClassifier()
# Entrenar el clasificador con los datos de entrenamiento
modeloDT.fit(X_train, y_train)
predicciones = modeloDT.predict(X_test)
# Evaluamos el modelo
puntaje = modeloDT.score(X_test, y_test)
# Imprimimos el puntaje obtenido
print("Puntaje: ", puntaje)
print(classification_report(y_test, predicciones, target_names=target_names, digits=4))

Puntaje:  0.953
              precision    recall  f1-score   support

     class 0     0.9495    0.9570    0.9532      1001
     class 1     0.9566    0.9489    0.9528       999

    accuracy                         0.9530      2000
   macro avg     0.9530    0.9530    0.9530      2000
weighted avg     0.9530    0.9530    0.9530      2000



In [15]:
# Crear un clasificador con RandomForest
modeloRF = RandomForestClassifier()
# Entrenar el clasificador con los datos de entrenamiento
modeloRF.fit(X_train, y_train)
predicciones = modeloRF.predict(X_test)
# Evaluamos el modelo
puntaje = modeloRF.score(X_test, y_test)
# Imprimimos el puntaje obtenido
print("Puntaje: ", puntaje)
print(classification_report(y_test, predicciones, target_names=target_names, digits=4))

Puntaje:  0.96
              precision    recall  f1-score   support

     class 0     0.9694    0.9500    0.9596      1001
     class 1     0.9509    0.9700    0.9604       999

    accuracy                         0.9600      2000
   macro avg     0.9602    0.9600    0.9600      2000
weighted avg     0.9602    0.9600    0.9600      2000



In [16]:
# Crear un clasificador con SVC
modeloSVC = SVC(kernel='linear', random_state=42)
# Entrenar el clasificador con los datos de entrenamiento
modeloSVC.fit(X_train, y_train)
predicciones = modeloSVC.predict(X_test)
# Evaluamos el modelo
puntaje = modeloSVC.score(X_test, y_test)
# Imprimimos el puntaje obtenido
print("Puntaje: ", puntaje)
print(classification_report(y_test, predicciones, target_names=target_names, digits=4))

Puntaje:  0.96
              precision    recall  f1-score   support

     class 0     0.9733    0.9461    0.9595      1001
     class 1     0.9474    0.9740    0.9605       999

    accuracy                         0.9600      2000
   macro avg     0.9603    0.9600    0.9600      2000
weighted avg     0.9604    0.9600    0.9600      2000



In [17]:
# Crear un clasificador con AdaBoostClassifier
modeloADA = AdaBoostClassifier()
# Entrenar el clasificador con los datos de entrenamiento
modeloADA.fit(X_train, y_train)
predicciones = modeloADA.predict(X_test)
# Evaluamos el modelo
puntaje = modeloADA.score(X_test, y_test)
# Imprimimos el puntaje obtenido
print("Puntaje: ", puntaje)
print(classification_report(y_test, predicciones, target_names=target_names, digits=4))

Puntaje:  0.9595
              precision    recall  f1-score   support

     class 0     0.9863    0.9321    0.9584      1001
     class 1     0.9355    0.9870    0.9605       999

    accuracy                         0.9595      2000
   macro avg     0.9609    0.9595    0.9595      2000
weighted avg     0.9609    0.9595    0.9595      2000



In [18]:
# Crear un clasificador con KNeighborsClassifier
modeloKN = KNeighborsClassifier()
# Entrenar el clasificador con los datos de entrenamiento
modeloKN.fit(X_train, y_train)
predicciones = modeloKN.predict(X_test)
# Evaluamos el modelo
puntaje = modeloKN.score(X_test, y_test)
# Imprimimos el puntaje obtenido
print("Puntaje: ", puntaje)
print(classification_report(y_test, predicciones, target_names=target_names, digits=4))

Puntaje:  0.9435
              precision    recall  f1-score   support

     class 0     0.9431    0.9441    0.9436      1001
     class 1     0.9439    0.9429    0.9434       999

    accuracy                         0.9435      2000
   macro avg     0.9435    0.9435    0.9435      2000
weighted avg     0.9435    0.9435    0.9435      2000

